<a href="https://colab.research.google.com/github/aime-n/ic/blob/main/gcn_groundtruth_1_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/HaoWeiHe/Knowledge-Graph

Cloning into 'Knowledge-Graph'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 428 (delta 242), reused 97 (delta 31), pack-reused 0
Receiving objects: 100% (428/428), 7.67 MiB | 8.84 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [2]:
mv ./Knowledge-Graph/* .

In [3]:
from knowledgeGraph import get_entity
from knowledgeGraph import get_relation

In [6]:
import pandas as pd
ground_truth = pd.read_csv('https://raw.githubusercontent.com/aime-n/ic/main/Ground_truth.csv')
ground_truth.head(3)

,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated)
0,com.zentertain.photoeditor,gp:AOqpTOEW40L9WXqCjzq04bqaZImgMdzlczxIF3_ibs8...,1,May be i can check,NaN,NaN,NaN,NaN
1,com.zentertain.photoeditor,gp:AOqpTOF57AQPvmnCiWYurwLY-F2-mej25ON8RAFk-Ls...,1,It make me happy,NaN,NaN,NaN,NaN
2,com.zentertain.photoeditor,gp:AOqpTOHYdmt72q4tSD8TZ8A5fZQqGivlBkIbWuHuJMZ...,1,I have a normal phone and it made 1 of my pics...,NaN,pics,NaN,pics


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import networkx as nx

KG = nx.Graph()
source = []
target = []

for review in ground_truth['Sentence content']:
  for sentence in sent_tokenize(review):
    # print(sentence)
    nodes = get_entity(sentence)
    relation = get_relation(sentence)
    # print(nodes,relation)
    node1 = nodes[0]
    node2 = nodes[1]
    if len(node1) > 0 and len(node2) > 0:
      KG.add_edge(node1,node2,relation=relation)
      source.append(node1)
      target.append(node2)


data_edges = {'source': source, 'target': target}
df_edges = pd.DataFrame(data_edges)
df_edges.head(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
adj = nx.adjacency_matrix(KG).todense()
g_attr = KG.copy()

c = 0

for node_id, node_data in g_attr.nodes(data = True):
  matrix = adj[c]
  node_data["feature"] = matrix
  c += 1

In [ ]:
list_features = []
list_nodes = []

for node_name in g_attr.nodes:
  feature_node = g_attr.nodes[node_name]['feature'].tolist()
  list_features.append(feature_node[0])
  list_nodes.append(node_name)

data_features = {'name': list_nodes, 'feature': list_features}

In [ ]:
matrix_adj = pd.DataFrame(adj)
matrix_adj['index'] = list_nodes
matrix_adj = matrix_adj.set_index('index')

matrix_adj.columns = list_nodes
matrix_adj.head(3)

In [ ]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

In [ ]:
import stellargraph as sg
G = sg.StellarGraph(matrix_adj, df_edges)
print(G.info())

In [ ]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [ ]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

In [ ]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

In [ ]:
epochs = 50

In [ ]:
train_gen = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

In [ ]:
test_gen = FullBatchLinkGenerator(G_test, method="gcn")
test_flow = train_gen.flow(edge_ids_test, edge_labels_test)

In [ ]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)

In [ ]:
x_inp, x_out = gcn.in_out_tensors()

In [ ]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

In [ ]:
prediction = keras.layers.Reshape((-1,))(prediction)

In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["binary_accuracy"],
)

In [ ]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
history = model.fit(
    train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))